In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 09:23:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.14136976


In [4]:
from pyspark.sql import SparkSession

#initialize spark session
spark = SparkSession.builder.appName("WordCount").getOrCreate()

In [5]:
text_file = spark.sparkContext.textFile("/home/labuser/Desktop/testfile.txt")

In [6]:
#Split lines into words abd flatten them
words = text_file.flatMap(lambda line: line.split(" "))

#Map each word to (word, 1) to prepare for counting
word_counts = words.map(lambda word: (word, 1))

#Reduce by key to count the occurences of each word
word_count = word_counts.reduceByKey(lambda a, b: a + b)

#collect the results
results = word_count.collect()

Py4JJavaError: An error occurred while calling o59.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/labuser/Desktop/testfile.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scala:61)
	

In [8]:
rdd = spark.sparkContext.parallelize([1, 23 , 4, 5])

In [9]:
rdd.collect()

[1, 23, 4, 5]

In [10]:
resultrdd = rdd.map(lambda x: x*2)

resultrdd.collect()

[2, 46, 8, 10]

In [11]:
wordlist=['this','is','a','sample','text','document','for','word','count','example','word','count']
rdd=spark.sparkContext.parallelize(wordlist)
wordcount=rdd.map(lambda word:(word,1)).groupByKey().mapValues(sum)

 

wordcount.collect()

[('this', 1),
 ('sample', 1),
 ('text', 1),
 ('for', 1),
 ('word', 2),
 ('is', 1),
 ('a', 1),
 ('document', 1),
 ('count', 2),
 ('example', 1)]

In [12]:
purchaserdd = spark.sparkContext.textFile("/home/labuser/Desktop/purchases.csv")

In [13]:
purchaserdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [14]:
purchasedf = spark.read.csv("/home/labuser/Desktop/purchases.csv")

In [15]:
purchasedf.show()

+------+------+-------+
|   _c0|   _c1|    _c2|
+------+------+-------+
|  null|apples|oranges|
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [16]:
purchasedf_01 = spark.read.option("inferSchema", True).option("Header", True).csv("/home/labuser/Desktop/purchases.csv")
purchasedf_01.show()
#Not good approach. Takes a lot of time because it reads the file twice.

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/21 09:25:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Desktop/purchases.csv


In [17]:
purchasedf.printSchema()
purchasedf_01.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

root
 |-- _c0: string (nullable = true)
 |-- apples: integer (nullable = true)
 |-- oranges: integer (nullable = true)



In [22]:
movie_review_df = spark.read.option("inferSchema", True).option("header", True).csv("/home/labuser/Desktop/IMDB-Movie-Data.csv")

In [23]:
movie_review_df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [25]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

udfschema = StructType([StructField("Rank", IntegerType(), True),
                        StructField("Title", StringType(), True),
                        StructField("Genre", StringType(), True)])

In [26]:
movie_review_df = spark.read.schema(udfschema).option("header", True).csv("/home/labuser/Desktop/IMDB-Movie-Data.csv")

In [27]:
movie_review_df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)

